In [98]:
import wandb
import random

from tqdm.auto import tqdm

import pandas as pd
import numpy as np

from matplotlib import pyplot as plt

In [99]:
api = wandb.Api()
wandb.login()

True

In [100]:
l_runs = [run for run in api.runs(path="dyck-benchmark/uncategorized") if run.state == 'finished']

In [101]:
runs = []

for run in tqdm(l_runs, desc="Loading and extracting config and history from runs"):
    keys = ["M", "k", "num_heads", "hidden_size", "num_layers", "word_length"]
    config = run.config
    
    if any([key not in config for key in keys]):
        continue

    summary = {k: int(v) for k, v in run.config.items() if k in keys}
    history = run.history()
    
    # drop irrelevant runs
    if summary["num_heads"] != 4:
        continue
    
    if "Test_accuracy" not in history or "Validation_accuracy" not in history:
        continue

    summary["test_acc"] = max(history["Test_accuracy"].dropna())
    summary["val_acc"] = max(history["Validation_accuracy"].dropna())

    runs.append(summary)

runs = pd.DataFrame(runs)
runs.head()

Loading and extracting config and history from runs:   0%|          | 0/211 [00:00<?, ?it/s]

,M,k,num_heads,num_layers,hidden_size,word_length,test_acc,val_acc
0,1,15,4,5,32,512,0.9685,0.9745
1,1,15,4,3,32,1024,0.9745,0.9930
2,1,5,4,5,32,1024,0.9930,0.9980
3,1,5,4,5,32,512,0.9960,0.9990
4,1,5,4,5,32,256,0.9890,0.9955


In [ ]:
from itertools import product

M_k_combos = list(product([1, 2, 5], [1, 5, 15]))

fig, axs = plt.subplots(3, 3, sharey=True, figsize=(10,10))

metric = "test_acc"

for i, (combo, ax) in enumerate(zip(M_k_combos, axs.flat)):
    M, k = combo

    df = runs.where((runs["M"].astype(int) == M) & (runs["k"].astype(int) == k)).dropna()
    df = df.sort_values([col for col in df.columns])

    for i_d, d in enumerate([16, 32, 64]):
      df_x = df[df["hidden_size"] == d]
      vals_3_256 = vals_3_512 = vals_3_1024 = 0
      vals_5_256 = vals_5_512 = vals_5_1024 = 0

      # want 6 values per hidden size: namely 3 for every num_layers in [3, 5]
      # do num_layers == 3
      df_3 = df_x[df_x["num_layers"] == 3.0]
      if len(df_3) > 0:
          if len(df_3[df_3["word_length"] == 256.0]) > 0:
              vals_3_256 = df_3[df["word_length"] == 256.0][metric].tolist()[0]
          if len(df_3[df_3["word_length"] == 512.0]) > 0:
              vals_3_512 = df_3[df["word_length"] == 512.0][metric].tolist()[0]
          if len(df_3[df_3["word_length"] == 1024.0]) > 0:
              vals_3_1024 = df_3[df["word_length"] == 1024.0][metric].tolist()[0]

      # do num_layers = 5
      df_5 = df_x[df_x["num_layers"] == 5.0]
      if len(df_5) > 0:
          if len(df_5[df["word_length"] == 256.0]) > 0:
              vals_5_256 = df_5[df_5["word_length"] == 256.0][metric].tolist()[0]
          if len(df_5[df["word_length"] == 512.0]) > 0:
              vals_5_512 = df_5[df_5["word_length"] == 512.0][metric].tolist()[0]
          if len(df_5[df["word_length"] == 1024.0]) > 0:
              vals_5_1024 = df_5[df_5["word_length"] == 1024.0][metric].tolist()[0]

      # print(vals_3_256, vals_3_512, vals_3_1024)
      # print(vals_5_256, vals_5_512, vals_5_1024)
      ax.bar(i_d - 0.3, vals_3_256, 0.1, label='L=3, n=256', color="tab:blue")
      ax.bar(i_d - 0.2, vals_3_512, 0.1, label='L=3, n=512', color="tab:orange")
      ax.bar(i_d - 0.1, vals_3_1024, 0.1, label='L=3, n=1024', color="tab:green")

      ax.bar(i_d + 0.1, vals_5_256, 0.1, label='L=5, n=256', color="tab:red")
      ax.bar(i_d + 0.2, vals_5_512, 0.1, label='L=5, n=512', color="tab:purple")
      ax.bar(i_d + 0.3, vals_5_1024, 0.1, label='L=5, n=1024', color="tab:brown")
    
    ax.set_xlim([-1, 3])
    ax.set_xticks([0, 1, 2], ["16", "32", "64"])
    ax.set_title(f"M={M} | k={k}")
    
    if i == 3:
        ax.set_ylabel(metric)
    if i == 7:
        ax.set_xlabel("Hidden Size")

handles, labels = plt.gca().get_legend_handles_labels()
by_label = dict(zip(labels, handles))
axs.flat[0].legend(by_label.values(), by_label.keys(), loc="lower right")

plt.tight_layout()
plt.show()

Plot Dimensions: 46 4
